In [40]:
from pyspark.sql import SparkSession
from delta import *

import subprocess
import json
import os

In [42]:
from uuid import uuid4

In [43]:
session_id = str(uuid4())

In [44]:
builder = SparkSession.builder.appName(session_id) \
    .master("spark://spark-master:7077")\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()    
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b1e24cd9-e5db-4683-92f3-75d2feda1607;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 482ms :: artifacts dl 18ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |  

In [57]:
from confluent_kafka.admin import AdminClient, NewTopic
from confluent_kafka import Producer, Consumer, KafkaError
from confluent_kafka.schema_registry import SchemaRegistryClient, Schema
from confluent_kafka import avro
import pyspark.sql.functions as pyspark_f
import pyspark.sql.types as pyspark_dt

# Create Topic & Schema Registry

In [56]:
topic_name = 'sample_topic'
kafka_config = {
    'bootstrap.servers': 'kafka-1:9092,kafka-2:9092,kafka-3:9092',
}

schema_registry_cofig =  {'url' : 'http://schema-registry:8085'} # URL for Schema Registry


In [55]:
# Create Kafka topic
admin_client = AdminClient(kafka_config)
new_topic = NewTopic(topic_name, num_partitions=2, replication_factor=2)

# Create the topic
admin_client.create_topics([new_topic])
print(f"Kafka topic {topic_name} created.")

Kafka topic sample_topic created.


In [ ]:
# Define the Avro schema
value_schema_str = '''
{
    "namespace": "example.avro",
    "type": "record",
    "name": "User",
    "fields": [
        {"name": "name", "type": "string"},
        {"name": "age", "type": "int"}
    ]
}
'''


# Register the Avro schema with the Schema Registry
schema_registry_client = SchemaRegistryClient(schema_registry_cofig)
value_serializer = avro.AvroSerializer(schema_registry_client, value_schema_str)
producer = avro.AvroProducer(kafka_config, default_value_schema=value_schema)

/tmp/ipykernel_7565/3896567996.py:17: DeprecationWarning: AvroProducer has been deprecated. Use AvroSerializer instead.
  producer = avro.AvroProducer(kafka_config, default_value_schema=value_schema)
